In [36]:
# ✅ basic
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.7)
chat.predict("hello? I'am Jenny What's your name?")

'Hello Jenny, I am an AI assistant. Nice to meet you! How can I help you today?'

In [ ]:
# ✅ messages
from langchain.schema import HumanMessage, AIMessage, SystemMessage
messages = [
            SystemMessage(content = "you are Korean. You speak mixing English and Korean" )
            , AIMessage(content="I'm John")
            , HumanMessage(content="Nice to meet you John!")
            ]

chat.predict_messages(messages)

AIMessage(content='반가워요! Nice to meet you too! How can I help you today?')

In [ ]:
# ✅ prompt template

from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template ("what is distance between {country_a} and {country_b}?")

prompt = template.format(country_a="Korea", country_b="USA")

chat.predict(prompt)

'The distance between Korea and the USA varies depending on the specific locations being compared. For example, the distance between Seoul, Korea and Los Angeles, USA is approximately 6,000 miles (9,656 kilometers) when measured in a straight line. However, the distance can be longer or shorter depending on the specific cities or regions being considered.'

In [ ]:
# ✅ prompt template 

template = PromptTemplate.from_template ("what bible verse do you reccommand when I am in situation {situation}?")

prompt = template.format(situation="happy beacause I got a job")

chat.predict(prompt)

'One Bible verse that may be fitting for a situation where you are feeling happy and grateful for receiving a job is Philippians 4:13, which states, "I can do all things through Christ who strengthens me." This verse serves as a reminder that God is with you and will give you the strength and abilities needed to succeed in your new job.'

In [ ]:
# ✅ chat promt template
template = ChatPromptTemplate.from_messages([
            ("system","you are {a}. You speak {language}" ),
            ("ai", "you soy John"),
            ("human","Nice to meet you John!")
            ])
prompt = template.format_messages(a="Arabic", language="Arabic")
chat.predict_messages(prompt)

AIMessage(content='مرحبا! سعيد بلقائك أيضًا! كيف يمكنني مساعدتك اليوم؟')

In [ ]:
# ✅ output parser
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, output):
        items = output.strip().split(",")
        return list(map(str.strip, items))
      
p = CommaOutputParser()
p.parse("a,  b,c")


['a', 'b', 'c']

In [40]:
# ✅ chat prompt template + parser

template = ChatPromptTemplate.from_messages([
  ("system" , "너는 이사람의 {situation}에 맞는 성경을 추천해줘야하는데 리스트로 반환해줘 여러절도 괜찮아 괜찮아 반환하는 결과가 첫번째 아이템은 제목 n장 m~l 절, 두번째 아이템은 본문구절, 마지막으로는 추가적으로 해주고 싶은 말 이렇게 세가지를 담은 리스트로 부탁해"),
  ("ai","('[ 마가복음 9장 23-24절]', '할 수 있거든이 무슨 말이냐, 믿는 자에게는 능치 못할 일이 없느니라 ','이 말씀을 너의 맘에 새기며 주님과 함께 집중해가길')"),
  ("human" , "나 이직준비를하는데 다른 재밌는것들이 많이 보여서 집중이 잘안돼 이럴때 어떤 성경구절을 읽으면 좋을까?")
  ])

prompt = template.format_messages(situation="이직준비를하는데 다른 재밌는것들이 많이 보여서 집중이 잘안돼 이럴때 어떤 성경구절을 읽으면 좋을까?")
chat.predict_messages(prompt)


AIMessage(content="['[ 빌립보서 4장 8절]', '무엇에든지 참되로요, 무엇에든지 도리어, 무엇에든지 의로우, 무엇에든지 정결하, 무엇에든지 사랑 받는 것, 무슨 덕 있으면 이것을 생각하라.', '하나님의 뜻을 따르며 그분과의 관골을 유지해가며 새로운 도전을 위해 힘을 내는 것도 좋은 방법이 될 것이다. 힘내세요!']")

In [52]:
# ✅ chat prompt template + parser

template = ChatPromptTemplate.from_messages([
  ("system" , "only return list. list length is {max_item}. and split with comma, if you don't know, just return empty list"),
  ("human" , "{question}")
  ])

prompt = template.format_messages(max_item=3,question="tell me country name to travel")
chat.predict_messages(prompt)


AIMessage(content='Australia, Japan, Italy')

In [ ]:
# ✅ chain : more efficient way to use langchain expression
chain = template | chat | CommaOutputParser() # chat predict -> output parser method 호출이 자동으로 됨
chain.invoke({"max_item": 3, "question": "tell me animal names"})

['cat', 'dog', 'rabbit']

In [ ]:
from langchain.callbacks import StreamingStdOutCallbackHandler
# ✅ chain 
chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
chef_prompt = ChatPromptTemplate.from_messages([
  ("system","You are a world-class international chef. You create easy to followe recipes for any types of cuisin with easy to find ingredients"),
  ("human","I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat 

# 내가 말하는 요리법을 채식 요리로 변경을 해주는 프롬프트
veg_chef_prompt = ChatPromptTemplate.from_messages([
  ("system","You speaks only Korean, You are a vegeterian chef specialized on making traditional recipies vegitarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
  ("human","{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = { "recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "Korean"})


Great choice! Here's a simple recipe for Bibimbap, a popular Korean dish that consists of mixed rice with vegetables, meat, and a spicy sauce. 

Ingredients:
- 1 cup of cooked white rice
- 1 carrot, julienned
- 1 zucchini, julienned
- 1 cup of bean sprouts
- 1 cup of spinach
- 1/2 pound of beef (you can use ground beef or thinly sliced beef)
- 2 cloves of garlic, minced
- 2 tablespoons of soy sauce
- 1 tablespoon of sesame oil
- 1 tablespoon of gochujang (Korean red chili paste)
- 1 tablespoon of vegetable oil
- Salt and pepper to taste
- Sesame seeds for garnish
- Fried egg (optional)

Instructions:
1. Cook the white rice according to the package instructions and set aside.
2. In a small bowl, mix the minced garlic, soy sauce, sesame oil, and gochujang to make the sauce. Set aside.
3. Heat the vegetable oil in a pan over medium heat. Add the beef and cook until browned. Season with salt and pepper to taste. Set aside.
4. In the same pan, add a little more oil if needed and stir-fry th

AIMessageChunk(content='안녕하세요! Bibimbap 레시피를 공유해 주셔서 감사합니다. 이 레시피를 채식주의자용으로 변형해 볼까요?\n\n대체재료 및 대체 방법:\n- 쇠고기: 쇠고기 대신 쇠고기 대용품으로 대파나 버섯을 사용할 수 있습니다. 쇠고기 대신 대파를 사용할 경우, 대파를 얇게 썰어서 소금, 후추, 간장, 참기름으로 조리한 후 불고기처럼 활용할 수 있습니다.\n- 계란: 계란은 선택 사항이므로 생략하거나 대체재료로 토피를 사용할 수 있습니다. 토피는 두부를 채 썰어 소금, 후추, 간장, 참기름으로 볶아 계란 대용품으로 활용할 수 있습니다.\n\n그 외의 재료는 레시피에 나열된 대로 준비하고 조리하시면 됩니다. Bibimbap을 즐기세요!')